# Code reference for pretrained models
https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/

# You can change the number of epoch in the training process to get better accuraces,but here for running time it set it to 1 

# Check the tensorflow GPU and torch GPU

In [ ]:
import pandas as pd
import tensorflow as tf
print(len(tf.config.list_physical_devices('GPU')))
import torch
print(torch.cuda.is_available())

# Import important libraries

In [ ]:

import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from skimage import transform

In [ ]:
photo_size=200

# Prepare dataset and add some augmentation

In [ ]:
def prepare_dataset(data_dir):
    datagen = ImageDataGenerator(
        rescale=1 / 255,
        rotation_range=40,
        width_shift_range=.2,
        height_shift_range=.2,
        shear_range=.1,
        horizontal_flip=True,
        fill_mode='nearest',
        zoom_range=.2,
    )
    generator = datagen.flow_from_directory(
        data_dir,
        target_size=(photo_size,photo_size),
        class_mode='binary',
        batch_size=128,
        # classes=[str(i) for i in range(2)]
    )
    return generator

In [ ]:
import zipfile
zip_df = zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip", 'r')
zip_df.extractall("/kaggle/working/")
zip_df.close()

# Spliting Dogs and Cats in the training file to determine the classes

In [ ]:
import os
from shutil import copyfile,move
file_path = "./train/"
os.mkdir(os.path.join(file_path, 'Cats'))
os.mkdir(os.path.join(file_path, 'Dogs'))
for i,image_name in enumerate(os.listdir(file_path)):
    if 'cat'in image_name :
        move(os.path.join(file_path, image_name),os.path.join(os.path.join(file_path, 'Cats'),image_name))

for i,image_name in enumerate(os.listdir(file_path)):
    if 'dog'in image_name :
        move(os.path.join(file_path, image_name),os.path.join(os.path.join(file_path, 'Dogs'),image_name))


# Create validation directory

In [ ]:
os.mkdir(os.path.join('./', 'validation'))


# Split training ,validation and  testing Dataset

In [ ]:
import shutil
import os
import numpy as np
import argparse

def get_files_from_folder(path):
    files = os.listdir(path)
    return np.asarray(files)

def image_train_test_split(path_to_data, path_to_test_data, train_ratio):
    # get dirs
    _, dirs, _ = next(os.walk(path_to_data))

    # calculates how many train data per class
    data_counter_per_class = np.zeros((len(dirs)))
    for i in range(len(dirs)):
        path = os.path.join(path_to_data, dirs[i])
        files = get_files_from_folder(path)
        data_counter_per_class[i] = len(files)
    test_counter = np.round(data_counter_per_class * (1 - train_ratio))

    # transfers files
    for i in range(len(dirs)):
        path_to_original = os.path.join(path_to_data, dirs[i])
        path_to_save = os.path.join(path_to_test_data, dirs[i])

        #creates dir
        if not os.path.exists(path_to_save):
            os.makedirs(path_to_save)
        files = get_files_from_folder(path_to_original)
        # moves data
        for j in range(int(test_counter[i])):
            dst = os.path.join(path_to_save, files[j])
            src = os.path.join(path_to_original, files[j])
            shutil.move(src, dst)



In [ ]:
image_train_test_split('./train/', './validation/', float(0.9))

# Load Dataset from directory

In [ ]:
train_data=prepare_dataset('./train/')
validation_data = prepare_dataset('./validation/')



# Unzip test data

In [ ]:
import zipfile
zip_df = zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip", 'r')
zip_df.extractall("/kaggle/working/")
zip_df.close()

In [ ]:
validation_data.class_indices

# Load pre-trained models

## Vgg model

In [ ]:
# example of tending the vgg16 model
def create_vgg_model():
    from keras.applications.vgg16 import VGG16
    from keras.models import Model
    from keras.layers import Dense
    from keras.layers import Flatten
    # load model without classifier layers
    model = VGG16(include_top=False, input_shape=(photo_size, photo_size, 3))
    for layer_idx in range(len(model.layers)):
        if layer_idx not in [1,2,3,15,16,17,18]:
            model.layers[layer_idx].trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu')(flat1)
    # class1 = Dense(256, activation='relu')(class1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    return model

In [ ]:
vgg_model=create_vgg_model()
vgg_model.summary()

In [ ]:
vgg_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
vgg_model.fit(
    train_data,
    epochs=1,
    validation_data=validation_data
)
vgg_model.save("Dogs_vs_cats_vgg_model30.h5")

## Inception model

In [ ]:
def create_inception_model():
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    base_model = InceptionV3(input_shape = (photo_size, photo_size, 3), include_top = False, weights = 'imagenet')
    for layer in base_model.layers:
        layer.trainable = False
    # for layer_idx in range(len(pretrained_model.layers)):
    #     if layer_idx not in [1,2,3,305,306,307,308,309,310]:
    #         pretrained_model.layers[layer_idx].trainable = False
    from tensorflow.keras.optimizers import RMSprop
    from tensorflow.keras import layers
    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(256, activation='relu')(x)
    # Add a final sigmoid layer with 1 node for classification output
    x = layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.models.Model(base_model.input, x)
    model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
inception_model=create_inception_model()
inception_model.summary()

In [ ]:
inception_model.fit(
    train_data,
    epochs=1,
    validation_data=validation_data,steps_per_epoch=5
)
inception_model.save("Dogs_vs_cats_inception_model_1_5step.h5")

In [ ]:
!pip install efficientnet

# Efficient net

In [ ]:
def use_efficient_net(model_type='B0'):
    from tensorflow.keras.optimizers import RMSprop
    from tensorflow.keras.models import Model
    from efficientnet.tfkeras import EfficientNetB0,EfficientNetB7
    if model_type=='B0':
        efn_model = EfficientNetB0(input_shape = (photo_size, photo_size, 3), include_top = False, weights = 'imagenet')
    else:
        efn_model = EfficientNetB7(input_shape = (photo_size, photo_size, 3), include_top = False, weights = 'imagenet')
    for layer in efn_model.layers:
        layer.trainable = False
    #
    x = efn_model.output
    x = Flatten()(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation="relu")(x)
    # Add a final sigmoid layer with 1 node for classification output
    predictions = Dense(1, activation="sigmoid")(x)
    efficient_net = Model(efn_model.input,predictions)

    efficient_net.compile(RMSprop(learning_rate=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])
    return efficient_net
efficient_net=use_efficient_net('B0')
efficient_net.summary()

In [ ]:
eff_history = efficient_net.fit(train_data, validation_data = validation_data, epochs = 1,steps_per_epoch=3)
efficient_net.save("Dog_vs_cats_efficient_netB0_1_3step.h5")

# Champion model Inference

In [ ]:
!pip install efficientnet

In [ ]:
from efficientnet.tfkeras import EfficientNetB0
from keras.models import load_model
efficient_net_model= load_model("Dogs_vs_cats_inception_model_1_5step.h5")
efficient_net_model.summary()

## Load Images from path

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
photo_size=200
def load_image_from_path(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32') / 255
    np_image = transform.resize(np_image, (photo_size, photo_size, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [ ]:
mTestPath = "test/"
submission={'id':[],'label':[]}
for i,test in enumerate(os.listdir(mTestPath)):
    img = load_image_from_path(os.path.join(mTestPath, test))
    submission['id'].append(i+1)
    res = efficient_net_model.predict(img)[0][0]
    submission['label'].append(res)
    # print(test, "\t", res)

In [ ]:
import pandas as pd
submission_df=pd.DataFrame(submission)
submission_df.to_csv('Dogs_vs_cats_inception_model_1_5step.csv',index=False)